In [1]:
import time, sys, argparse
sys.path.append('../../../../../')
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from dualbound.Maxwell import TM_FDFD as TM
from objective_TM_ext import ext_objective

def visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name='testcloak.png', pml_sep=3.0, binarize_des=False, angle=0.0):
    parser = argparse.ArgumentParser()
    parser.add_argument('-wavelength',action='store',type=float,default=1.0)
    #separation between pml inner boundary and source walls
    parser.add_argument('-pml_thick',action='store',type=float,default=0.5)

    args,unknown = parser.parse_known_args()
    print('the parameters for this run:', flush=True)
    for arg in vars(args):
        print(arg,'is',getattr(args,arg), flush=True)

    wv = args.wavelength
    omega = 2*np.pi/wv*(1 + 1j/2/Qabs)
    omega0 = 2*np.pi/wv
    do_checks = False

    pml_thick = args.pml_thick
    
    if binarize_des:
        design_dof = np.round(np.load(design_file))
    else:
        design_dof = np.load(design_file)
    
    des_background = np.load(background_file)
    plt.figure(figsize=((7,7)))
    plt.imshow(design_dof + des_background/2,cmap='Greys')
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.savefig(save_name+'_design.png', bbox_inches='tight')

    dl = 1.0/gpr
    Mx,My = des_background.shape 
    Npml = int(np.round(pml_thick/dl))
    Npmlsep = int(np.round(pml_sep/dl))
    nonpmlNx = Mx + 2*Npmlsep
    nonpmlNy = My + 2*Npmlsep
    Nx = nonpmlNx + 2*Npml
    Ny = nonpmlNy + 2*Npml
    
    #define the masks
    background_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) #the fixed object, which is considered to be part of background    
    #load background
    desMx,desMy = design_dof.shape 
    indxL = Npmlsep
    indxR = indxL + desMx
    indyL = Npmlsep
    indyR = indyL + desMy
    background_mask[indxL:indxR, indyL:indyR] = (des_background[:,:] > 0.0)
    design_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) 
    design_mask[indxL:indxR, indyL:indyR] = (design_dof[:,:] > 0.0) #will look odd for non-binarized results
        
    if do_checks:
        config = np.zeros((nonpmlNx,nonpmlNy))
        config[design_mask] = 1.0
        plt.figure()
        plt.imshow(config)
        plt.show()
        plt.savefig('check_design.png')
    
    if do_checks:
        config3 = np.zeros((nonpmlNx,nonpmlNy))
        config3[design_mask] = 1.0
        config3[background_mask] += 2.0
        plt.figure()
        plt.imshow(config3)
        plt.show()
        plt.savefig('check_design+background.png')
    
    background_chi = background_mask * chif #change from boolean mask of fixed object to complex numeric array with chi value
################# calculate the fixed vectors #################
    Z = 1.0 # dimensionless units
    C_0 = 1.0
    eps0 = 1/Z/C_0
    
    big_background_chi = np.zeros((Nx, Ny), dtype=complex) #background chi over grid including pml region
    big_background_chi[Npml:-Npml, Npml:-Npml] = background_chi[:,:]
        
    small_design_chi = np.zeros((nonpmlNx,nonpmlNy), dtype=complex) 
    small_design_chi[indxL:indxR, indyL:indyR] = design_dof[:,:]
    big_design_chi = np.zeros((Nx, Ny), dtype=complex)
    big_design_chi[Npml:-Npml, Npml:-Npml] = small_design_chi[:,:] #design mask over grid including pml region

    objcx = Npml + Npmlsep + Mx//2
    objcy =  Npml + Npmlsep + My//2
    #generate an initial plane wave.
    NpmlsepID = int(np.round(0.5/dl)) #Npmlsep used in inverse design run
    cx = Npml + Npmlsep + (-NpmlsepID + NpmlsepID//2) #position of current sheet correct dist away from cloak+obj as done in inverse design

    cx = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design
    cy = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design

    #field in vacuum
    Ei = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None)
    
    #Ei = Ei_cx + Ei_cy
    
    #field in vacuum
    Ei = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None)
    fig, axs = plt.subplots(2,3)
    axs[0,0].imshow(np.real(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    axs[1,0].imshow(np.imag(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
        
    #field in presence of fixed object only
    Ebowtie = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=big_background_chi)
    axs[0,1].imshow(np.real(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,1].add_patch(circle)
    
    axs[1,1].imshow(np.imag(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,1].add_patch(circle)
    
    #field in presence of designed cloak
    tot_chi = big_background_chi + (chid*big_design_chi)
    Etot = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=tot_chi)
    axs[0,2].imshow(np.real(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,2].add_patch(circle)
    
    axs[1,2].imshow(np.imag(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,2].add_patch(circle)
    
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])  
    plt.savefig(save_name+'_Etot_field.png',bbox_inches='tight')
    plt.close('all')

In [2]:
import time, sys, argparse
sys.path.append('../photonic-dual-bounds/')
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from dualbound.Maxwell import TM_FDFD as TM
from objective_TM_ext import ext_objective

def visualize_cloak_fields_cy(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name='testcloak.png', pml_sep=3.0, binarize_des=False, angle=0.0):
    parser = argparse.ArgumentParser()
    parser.add_argument('-wavelength',action='store',type=float,default=1.0)
    #separation between pml inner boundary and source walls
    parser.add_argument('-pml_thick',action='store',type=float,default=0.5)

    args,unknown = parser.parse_known_args()
    print('the parameters for this run:', flush=True)
    for arg in vars(args):
        print(arg,'is',getattr(args,arg), flush=True)

    wv = args.wavelength
    omega = 2*np.pi/wv*(1 + 1j/2/Qabs)
    omega0 = 2*np.pi/wv
    do_checks = False

    pml_thick = args.pml_thick
    
    if binarize_des:
        design_dof = np.round(np.load(design_file))
    else:
        design_dof = np.load(design_file)
    
    des_background = np.load(background_file)
    plt.figure(figsize=((7,7)))
    plt.imshow(design_dof + des_background/2,cmap='Greys')
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.savefig(save_name+'_design.png', bbox_inches='tight')

    dl = 1.0/gpr
    Mx,My = des_background.shape 
    Npml = int(np.round(pml_thick/dl))
    Npmlsep = int(np.round(pml_sep/dl))
    nonpmlNx = Mx + 2*Npmlsep
    nonpmlNy = My + 2*Npmlsep
    Nx = nonpmlNx + 2*Npml
    Ny = nonpmlNy + 2*Npml
    
    #define the masks
    background_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) #the fixed object, which is considered to be part of background    
    #load background
    desMx,desMy = design_dof.shape 
    indxL = Npmlsep
    indxR = indxL + desMx
    indyL = Npmlsep
    indyR = indyL + desMy
    background_mask[indxL:indxR, indyL:indyR] = (des_background[:,:] > 0.0)
    design_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) 
    design_mask[indxL:indxR, indyL:indyR] = (design_dof[:,:] > 0.0) #will look odd for non-binarized results
        
    if do_checks:
        config = np.zeros((nonpmlNx,nonpmlNy))
        config[design_mask] = 1.0
        plt.figure()
        plt.imshow(config)
        plt.show()
        plt.savefig('check_design.png')
    
    if do_checks:
        config3 = np.zeros((nonpmlNx,nonpmlNy))
        config3[design_mask] = 1.0
        config3[background_mask] += 2.0
        plt.figure()
        plt.imshow(config3)
        plt.show()
        plt.savefig('check_design+background.png')
    
    background_chi = background_mask * chif #change from boolean mask of fixed object to complex numeric array with chi value
################# calculate the fixed vectors #################
    Z = 1.0 # dimensionless units
    C_0 = 1.0
    eps0 = 1/Z/C_0
    
    big_background_chi = np.zeros((Nx, Ny), dtype=complex) #background chi over grid including pml region
    big_background_chi[Npml:-Npml, Npml:-Npml] = background_chi[:,:]
        
    small_design_chi = np.zeros((nonpmlNx,nonpmlNy), dtype=complex) 
    small_design_chi[indxL:indxR, indyL:indyR] = design_dof[:,:]
    big_design_chi = np.zeros((Nx, Ny), dtype=complex)
    big_design_chi[Npml:-Npml, Npml:-Npml] = small_design_chi[:,:] #design mask over grid including pml region

    objcx = Npml + Npmlsep + Mx//2
    objcy =  Npml + Npmlsep + My//2
    #generate an initial plane wave.
    NpmlsepID = int(np.round(0.5/dl)) #Npmlsep used in inverse design run
    cx = Npml + Npmlsep + (-NpmlsepID + NpmlsepID//2) #position of current sheet correct dist away from cloak+obj as done in inverse design

    cx = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design
    cy = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design

    #field in vacuum
    Ei = TM.get_TM_linesource_field_cy(wv, dl, Nx, Ny, cy, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None, angle=angle)
    
    #Ei = Ei_cx + Ei_cy
    
    #field in vacuum
    Ei = TM.get_TM_linesource_field_cy(wv, dl, Nx, Ny, cy, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None, angle=angle)
    fig, axs = plt.subplots(2,3)
    axs[0,0].imshow(np.real(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    axs[1,0].imshow(np.imag(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
        
    #field in presence of fixed object only
    Ebowtie = TM.get_TM_linesource_field_cy(wv, dl, Nx, Ny, cy, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=big_background_chi, angle=angle)
    axs[0,1].imshow(np.real(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,1].add_patch(circle)
    
    axs[1,1].imshow(np.imag(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,1].add_patch(circle)
    
    #field in presence of designed cloak
    tot_chi = big_background_chi + (chid*big_design_chi)
    Etot = TM.get_TM_linesource_field_cy(wv, dl, Nx, Ny, cy, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=tot_chi, angle=angle)
    axs[0,2].imshow(np.real(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,2].add_patch(circle)
    
    axs[1,2].imshow(np.imag(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,2].add_patch(circle)
    
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])  
    plt.savefig(save_name+'_Etot_field.png',bbox_inches='tight')
    plt.close('all')

In [3]:
import time, sys, argparse
sys.path.append('../photonic-dual-bounds/')
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from dualbound.Maxwell import TM_FDFD as TM
from objective_TM_ext import ext_objective

def visualize_cloak_fields_big_angle(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name='testcloak.png', pml_sep=3.0, binarize_des=False, angle=0.0):
    parser = argparse.ArgumentParser()
    parser.add_argument('-wavelength',action='store',type=float,default=1.0)
    #separation between pml inner boundary and source walls
    parser.add_argument('-pml_thick',action='store',type=float,default=0.5)

    args,unknown = parser.parse_known_args()
    print('the parameters for this run:', flush=True)
    for arg in vars(args):
        print(arg,'is',getattr(args,arg), flush=True)

    wv = args.wavelength
    omega = 2*np.pi/wv*(1 + 1j/2/Qabs)
    omega0 = 2*np.pi/wv
    do_checks = False

    pml_thick = args.pml_thick
    
    if binarize_des:
        design_dof = np.round(np.load(design_file))
    else:
        design_dof = np.load(design_file)
    
    des_background = np.load(background_file)
    plt.figure(figsize=((7,7)))
    plt.imshow(design_dof + des_background/2,cmap='Greys')
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.savefig(save_name+'_design.png', bbox_inches='tight')

    dl = 1.0/gpr
    Mx,My = des_background.shape 
    Npml = int(np.round(pml_thick/dl))
    Npmlsep = int(np.round(pml_sep/dl))
    nonpmlNx = Mx + 2*Npmlsep
    nonpmlNy = My + 14*Npmlsep #make longer in y direction
    Nx = nonpmlNx + 2*Npml
    Ny = nonpmlNy + 2*Npml
    
    #define the masks
    background_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) #the fixed object, which is considered to be part of background    
    #load background
    desMx,desMy = design_dof.shape 
    indxL = Npmlsep
    indxR = indxL + desMx
    indyL = Npmlsep
    indyR = indyL + desMy
    background_mask[indxL:indxR, indyL:indyR] = (des_background[:,:] > 0.0)
    design_mask = np.zeros((nonpmlNx,nonpmlNy), dtype=bool) 
    design_mask[indxL:indxR, indyL:indyR] = (design_dof[:,:] > 0.0) #will look odd for non-binarized results
        
    if do_checks:
        config = np.zeros((nonpmlNx,nonpmlNy))
        config[design_mask] = 1.0
        plt.figure()
        plt.imshow(config)
        plt.show()
        plt.savefig('check_design.png')
    
    if do_checks:
        config3 = np.zeros((nonpmlNx,nonpmlNy))
        config3[design_mask] = 1.0
        config3[background_mask] += 2.0
        plt.figure()
        plt.imshow(config3)
        plt.show()
        plt.savefig('check_design+background.png')
    
    background_chi = background_mask * chif #change from boolean mask of fixed object to complex numeric array with chi value
################# calculate the fixed vectors #################
    Z = 1.0 # dimensionless units
    C_0 = 1.0
    eps0 = 1/Z/C_0
    
    big_background_chi = np.zeros((Nx, Ny), dtype=complex) #background chi over grid including pml region
    big_background_chi[Npml:-Npml, Npml:-Npml] = background_chi[:,:]
        
    small_design_chi = np.zeros((nonpmlNx,nonpmlNy), dtype=complex) 
    small_design_chi[indxL:indxR, indyL:indyR] = design_dof[:,:]
    big_design_chi = np.zeros((Nx, Ny), dtype=complex)
    big_design_chi[Npml:-Npml, Npml:-Npml] = small_design_chi[:,:] #design mask over grid including pml region

    objcx = Npml + Npmlsep + Mx//2
    objcy =  Npml + Npmlsep + My//2
    #generate an initial plane wave.
    NpmlsepID = int(np.round(0.5/dl)) #Npmlsep used in inverse design run
    cx = Npml + Npmlsep + (-NpmlsepID + NpmlsepID//2) #position of current sheet correct dist away from cloak+obj as done in inverse design

    cx = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design
    cy = Npml + Npmlsep//2 #position of current sheet correct dist away from cloak+obj as done in inverse design

    #field in vacuum
    Ei = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None)
    
    #Ei = Ei_cx + Ei_cy
    
    #field in vacuum
    Ei = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=None)
    fig, axs = plt.subplots(2,3)
    axs[0,0].imshow(np.real(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    axs[1,0].imshow(np.imag(Ei[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
        
    #field in presence of fixed object only
    Ebowtie = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=big_background_chi)
    axs[0,1].imshow(np.real(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,1].add_patch(circle)
    
    axs[1,1].imshow(np.imag(Ebowtie[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,1].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,1].add_patch(circle)
    
    #field in presence of designed cloak
    tot_chi = big_background_chi + (chid*big_design_chi)
    Etot = TM.get_TM_linesource_field_cx_angle(wv, dl, Nx, Ny, cx, angle, Npml, bloch_x=0.0, bloch_y=0.0, amp=1.0, Qabs=Qabs, chigrid=tot_chi)
    axs[0,2].imshow(np.real(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circcx = Mx//2 + Npmlsep//2
    circcy = My//2 + Npmlsep//2
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[0,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[0,2].add_patch(circle)
    
    axs[1,2].imshow(np.imag(Etot[objcx - Mx//2 - Npmlsep//2:objcx + Mx//2 + Npmlsep//2, objcy - My//2 - Npmlsep//2:objcy + My//2 + Npmlsep//2]), cmap='RdBu')
    
    circle = plt.Circle((circcx, circcy), (0.25/dl), color='lime', fill=False)
    axs[1,2].add_patch(circle)
    circle = plt.Circle((circcx, circcy), (design_x/dl)/2.0, color='lime', fill=False)
    axs[1,2].add_patch(circle)
    
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])  
    plt.savefig(save_name+'_Etot_field.png',bbox_inches='tight')
    plt.close('all')

In [ ]:
design_file = 'results/cloak_Rinner0d25_Router0d75_chif4+1e-3j_chid2+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Rinner0d25_Router0d75_chif4+1e-3j_chid2+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Rinner0d25_Router0d75_chif4+1e-3j_chid2+1e-3j_Qabs5.0e+06_L1.5_optdof'
Qabs = 5e6
chif = 4 + 1e-3j
chid = 2 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=1.0, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'test_cloak'
Qabs = 5e6
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=1.0, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'visualize_cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=1.0, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_gpr50_Rinner0d25_Router0d75_chifRe-4+1j_chidRe3+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_gpr50_Rinner0d25_Router0d75_chifRe-4+1j_chidRe3+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_gpr50_Rinner0d25_Router0d75_chifRe-4+1j_chidRe3+1e-3j_Qabs5.0e+06_L1.5_optdof'
Qabs = 5e6
chif = -4 + 1j
chid = 4 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=1.0, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_sweepchidIm_rand_gpr50_chif10+1e-1j_chidRe12_Rinner0d25_Router0d75_chidIm1.00000000e-06_L1.5_optdof.npy'
background_file = 'results/cloak_sweepchidIm_vac_gpr50_chif10+1e-1j_chidRe12_Rinner0d25_Router0d75_L1.5_backgrounddof.npy'
save_name = 'cloak_sweepchidIm_rand_gpr50_chif10+1e-1j_chidRe12_Rinner0d25_Router0d75_chidIm1.00000000e-06_L1.5_optdof'
Qabs = np.inf
chif = 10 + 1e-1j
chid = 12 + 1e-6j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_15degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=15)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_0degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=0.0)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_15degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=15)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_0degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=0.0)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_20degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=20)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_0degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=0.0)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'visualize_cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_15degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=15.0)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'visualize_cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_0degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=0.0)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_rand_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_90degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields_cy(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif-4+1j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_90degrees'
Qabs = np.inf
chif = -4 + 1j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields_cy(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'visualize_cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_90degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields_cy(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False)

In [ ]:
design_file = 'results/cloak_Qabs_sweep_rand_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'cloak_Qabs_sweep_vac_gpr50_circle_2sides_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+02_L1.5_optdof_90degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields_cy(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=10.0)

the parameters for this run:
wavelength is 1.0
pml_thick is 0.5


In [ ]:
design_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof.npy'
background_file = 'results/cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_L1.5_backgrounddof.npy'
save_name = 'visualize_cloak_Qabs_sweep_vac_gpr50_Rinner0d25_Router0d75_chif4+1e-3j_chid5+1e-3j_Qabs5.0e+06_L1.5_optdof_15degrees'
Qabs = np.inf
chif = 4 + 1e-3j
chid = 5 + 1e-3j
design_x = 1.5
design_y = 1.5
gpr = 50
visualize_cloak_fields_big_angle(Qabs, chif, chid, design_x, design_y, gpr, design_file, background_file, save_name, pml_sep=2.5, binarize_des=False, angle=45.0)